In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""

df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

## Preparing for training

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [8]:
x,y = data, labels
x_train, x_test, y_train, y_test = train_test_split(x, y)

### Build and train the TensorFlow model

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

Then we complile our model, and start training it.

- model optimizer, loss function and metrics.

Since this is a regression model (predicting a numerical value), we're using mean squared error instead of accuracy as our metric

In [12]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
             loss=tf.keras.losses.MeanSquaredError(),
             metrics=['mae', 'mse'])

Let's see the number of parameters for each layer

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 356us/sample - loss: 1.6678 - mae: 0.9913 - mse: 1.6678 - val_loss: 1.4749 - val_mae: 0.9488 - val_mse: 1.4749
Epoch 2/10
6670/6670 [==============================] - 1s 139us/sample - loss: 1.3607 - mae: 0.9162 - mse: 1.3607 - val_loss: 2.1723 - val_mae: 1.2044 - val_mse: 2.1723
Epoch 3/10
6670/6670 [==============================] - 1s 129us/sample - loss: 1.3302 - mae: 0.8994 - mse: 1.3302 - val_loss: 1.5246 - val_mae: 0.9629 - val_mse: 1.5246
Epoch 4/10
6670/6670 [==============================] - 1s 188us/sample - loss: 1.2905 - mae: 0.8894 - mse: 1.2905 - val_loss: 1.6738 - val_mae: 1.0276 - val_mse: 1.6738
Epoch 5/10
6670/6670 [==============================] - 1s 142us/sample - loss: 1.2854 - mae: 0.8904 - mse: 1.2854 - val_loss: 1.2232 - val_mae: 0.8560 - val_mse: 1.2232
Epoch 6/10
6670/6670 [==============================] - 1s 198us/sample - loss: 1.2504 - mae: 0.8738 - 

### Generate predictions on test examples

In [16]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [17]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ', y_test.iloc[i])
    print()

Predicted val:  8.152605
Actual val:  5.68351711436

Predicted val:  7.512957
Actual val:  8.313631900019999

Predicted val:  6.8482094
Actual val:  4.9383546688

Predicted val:  7.215439
Actual val:  7.31273323054

Predicted val:  6.6757545
Actual val:  6.0627122049999995

Predicted val:  7.3427787
Actual val:  7.1870697412

Predicted val:  7.4434204
Actual val:  8.375361333379999

Predicted val:  7.3745546
Actual val:  8.56275425608

Predicted val:  7.7086163
Actual val:  9.49971886958

Predicted val:  8.000798
Actual val:  8.6862131228



----

#### Prepare data for the What-If Tool

In [18]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [21]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [22]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
        .set_custom_predict_fn(custom_predict)
        .set_target_feature('weight_pounds')
        .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…

Explore model behavior with the What-If Tool

### Key points of this GCP lab

- Create an customize an AI Platform Notebook instance
- Initialize a local git repo in that instance, add commits via the git UI or command line, view git diffs in the Notebook git UI
- Build and train a simple TensorFlow 2 model
- Use the What-If Tool within your Notebook instance
- Connect your Notebook git repo to an external repository on GitHu